In [1]:
import sys
sys.path.insert(0,'..')

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import torch as th
import pandas as pd
import os
import matplotlib.pyplot as plt
import pyro.distributions as dist
import pyro

from bayes.parameters import ParameterList
from bayes.inference_problem import VariationalBayesProblem, ModelErrorInterface
from bayes.Calibration import Inference

# Aim
We have load-displacemnt RAW data as a function of time $t$ for a cylindrical concrete with daimeter $d$.

- Here the parameter to inferred (E) is not known beforehand
- Known Input : $\sigma(t)$
- Noisy Observed values : $\epsilon(t)$
- To infer : E (Youngs Modulus) and noise (treated as a gaussian and s.d is inferred)
    - Gamma Hyperprior for the noise term

From the experimental data E in the range ~E10

# 1. Reading in the data

In [3]:
path = 'usecases/youngsModulusConcrete/data/1datengefiltert.txt'
data = pd.read_csv(path,delimiter="\t",skiprows=3,skipfooter=125)

<ipython-input-3-2cfaf908e76d>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(path,delimiter="\t",skiprows=3,skipfooter=125)


In [ ]:
# Converting , to . so that import the values as float rather than string

data['mm.1'] = [x.replace(',', '.') for x in data['mm.1']]

data['mm.1'] = data['mm.1'].astype(float)
data['mm.2'] = [x.replace(',', '.') for x in data['mm.2']]

data['mm.2'] = data['mm.2'].astype(float)
data['mm.3'] = [x.replace(',', '.') for x in data['mm.3']]

data['mm.3'] = data['mm.3'].astype(float)
data['s'] = [x.replace(',', '.') for x in data['s']]

data['s'] = data['s'].astype(float)
data['kN.1'] = [x.replace(',', '.') for x in data['kN.1']]

data['kN.1'] = data['kN.1'].astype(float)

In [ ]:
data = data.drop(labels=range(0, 490), axis=0)
#data = data.drop(labels=range(600, 667), axis=0)

In [ ]:
length = 100 #As reported by Jorg
data['strain'] = ((data['mm.1'] + data['mm.2'] + data['mm.3'])/3)/length

dia=0.0985 #As per the .txt file
data['stress'] = (data['kN.1']*1000)/(np.pi*(dia/2)**2)

In [ ]:
data

In [ ]:
(np.max(data['stress'])-np.min(data['stress']))/(np.max(data['strain']-np.min(data['strain'])))

In [ ]:
plt.plot(data['strain'],data['stress'])
plt.xlabel('strain')
plt.ylabel('stress')
plt.title('Experimental Values for last load cycle')
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))

In [ ]:
# 70-30 training and test split
train, test = np.split(data.sample(frac=1, random_state=42), 
                       [int(.7*len(data))])

# 2. Forward Solve

In [ ]:
def forward_solve(known_input, latent_para):
    assert isinstance(known_input['known_inputs'],np.ndarray)
    stress = known_input['known_inputs']
    strain = th.tensor(stress)/latent_para
    
    return strain

# 3. Inference Task
- Uniform Prior for the latent variable E
- Gamm hyperprior for the noise term

In [ ]:
# -- Metadata for Inference problem
        
prior_hyperparameter = [32E09, 38E09]
prior_dist = "Uniform"
Observed_data = th.tensor(train['strain'].to_numpy())
Noise_distribution = "Normal"
Noise_hyperparameter = None  # TODO: test with correlated noise model
HyperPrior_dist = "Gamma"
Hyperprior_Parameter = [1,1E05]

 # ---- Metadata for forward solve
forward_solve_wrapper = forward_solve
forward_solve_known_input = train['stress'].to_numpy()
forward_input = {'known_inputs':forward_solve_known_input}

# -- Setup the Inference problem
infer = Inference(prior_dist, prior_hyperparameter, forward_solve_wrapper, forward_input, Observed_data,
                  Noise_distribution, Noise_hyperparameter,HyperPrior_dist,Hyperprior_Parameter)

# -- Solve the Inference problem
   
posterior_para, posterior_noise = infer.run(1000, kernel="NUTS")

In [ ]:
# The strain values are very small. Need to normalize? What noise hyperprior dist? Their parameters?

## 3.1 Inference Visualisation 

In [ ]:
infer.visualize_prior_posterior(posterior_para, posterior_noise)

# 4. Prediction

In [ ]:
# known input 
tmp = test['stress'].to_numpy() # AA: Can write a wrapper to read in data if they are in same order (maybe for similar experiment), can also imporrt data from different experiment of the similar specimen
new_input_forward = {'known_inputs': tmp}

til_X = infer.predict(posterior_para,posterior_noise,new_input_forward)

In [ ]:
pos=np.quantile(til_X,[0.05,0.5,0.95],axis=0)

In [ ]:
#plt.plot(data['s'],pos[1,:])
plt.ticklabel_format(style='sci', axis='x', scilimits=(0, 0))
#plt.fill_betweenx(test['stress'],pos[0,:],pos[2,:],alpha=0.8, label='Predicted strain')
#plt.fill_between(test['stress'],pos[0,:],pos[2,:],alpha=0.8, label='Predicted strain')
plt.plot(test['strain'],test['stress'],'g', label='Noisy Experiment')
plt.plot(pos[1,:],test['stress'],'b',alpha=0.2)
plt.plot(pos[0,:],test['stress'],'b',alpha=0.2,label='Prediction')
plt.plot(pos[2,:],test['stress'],'b',alpha=0.2)
plt.legend()
plt.xlabel('strain')
plt.ylabel('stress')

In [ ]:
np.shape(til_X)

In [ ]:
np.shape(pos)

# Misc

In [ ]:

sampled_parameters = pyro.sample('test', dist.Normal(0, 1))

In [ ]:
for name,value in sampled_parameters:
    print(name,value)

In [ ]:
sampled_parameters.items()